Parse input

In [ ]:
import pandas as pd

# Pentru eroarea cu tensorflow
import os
os.environ["USE_TF"] = "0"

from sentence_transformers import SentenceTransformer, util

companies = pd.read_csv('ml_insurance_challenge.csv')
description = companies['description'].tolist()
business_tags = companies['business_tags'].tolist()
sector = companies['sector'].tolist()
category = companies['category'].tolist()
niche = companies['niche'].tolist()

insurance_taxonomy = pd.read_csv('insurance_taxonomy.csv')
labels = insurance_taxonomy['label'].tolist()


Elimina randurile cu valori Nan in orice coloana, deoarece cand incarcam sa creez embeddings aveam erori

In [ ]:
companies = companies.dropna(subset=['description', 'business_tags', 'sector', 'category', 'niche'])

Verific numarul valorilor Nan din fiecare coloana

In [ ]:
print(companies.isna().sum())

Creez modelul NLP

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

Creez embeddings pentru textul din csv companii si csv taxonomy

In [11]:

companies['text'] = companies['description'] + ' ' + companies['business_tags'] + ' ' + companies['sector'] + companies['category'] + companies['niche']
company_embeddings = model.encode(companies['text'].tolist(), convert_to_tensor=True)
taxonomy_embeddings = model.encode(labels, convert_to_tensor=True)

Calculez similaritatea intre fiecare embedding al fiecarei companii cu taxonomy

In [ ]:

results = []
for emb in company_embeddings:
    cosine_scores = util.cos_sim(emb, taxonomy_embeddings)
    top_labels = [labels[i] for i in cosine_scores[0].topk(3).indices]  # top 3 labels
    results.append(top_labels)

Fac o noua coloana continand label urile si creez un nou csv din cel initial ar companiilor, cu aceasta coloana

In [13]:
companies['insurance_label'] = results

companies.to_csv('company_list_result.csv', index=False)